In [2]:
import os
os.chdir('..')
print(os.getcwd())

/Users/Placebo/OMSCS/CS7641-ML/MachineLearningProjects/UnsupervisedLearning


In [3]:
import numpy as np
import pandas as pd

In [55]:
%load_ext autoreload
%autoreload 2
from src.experiments.experiment4_RdNN import ExperimentRdNN, ExperimentNN
from src.utils.data_loader import load_processed_data, load_pickle
from src.utils.evaluation import organize_experiment4_results
from src.utils.plotting import plot_train_val_score, plot_learning_curves

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [4]:
dataset = 'dataset1'
experiment_name = 'experiment4'
X_train, X_test,y_train, y_test = load_processed_data(f'data/{dataset}')
print('The training data has shape: ', X_train.shape)

The training data has shape:  (32950, 30)


# Run Fine tuning Experiment

In [5]:
n_components_range = [8, 10, 12, 14, 16, 18, 20, 22]
params_grid = {
     'hidden_layer_sizes': [[5], [10], [15], [15, 10],  [10, 10], [10, 5]],
    'learning_rate_init': [0.1, 0.05, 0.02, 0.01, .005, .001, .0005, .0001]
}


## 1. PCA

In [15]:
exp = ExperimentRdNN(
    rd = 'pca',
    n_components_range = n_components_range,
    nn_params_grid=params_grid, 
    save_dir=f'results/{dataset}/{experiment_name}' 
)
results = exp.run_experiment(X_train, X_test, y_train, y_test)
# save
exp.save_results(results)


Running RD with n_components=8
Running NN
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Evaluating NN
[CV 4/5] END hidden_layer_sizes=[5], learning_rate_init=0.1;, score=(train=0.436, test=0.433) total time=   0.4s
[CV 3/5] END hidden_layer_sizes=[5], learning_rate_init=0.02;, score=(train=0.371, test=0.356) total time=   0.4s
[CV 3/5] END hidden_layer_sizes=[5], learning_rate_init=0.005;, score=(train=0.449, test=0.438) total time=   0.5s
[CV 4/5] END hidden_layer_sizes=[5], learning_rate_init=0.0005;, score=(train=0.471, test=0.472) total time=   0.5s
[CV 1/5] END hidden_layer_sizes=[10], learning_rate_init=0.1;, score=(train=0.401, test=0.362) total time=   0.6s
[CV 2/5] END hidden_layer_sizes=[10], learning_rate_init=0.05;, score=(train=0.398, test=0.409) total time=   0.3s
[CV 5/5] END hidden_layer_sizes=[10], learning_rate_init=0.05;, score=(train=0.472, test=0.466) total time=   0.6s
[CV 1/5] END hidden_layer_sizes=[10], learning_rate_init=0.005;, score=(train=0.

KeyboardInterrupt: 

## 2. ICA

In [7]:
exp = ExperimentRdNN(
    rd = 'ica',
    n_components_range = n_components_range,
    nn_params_grid=params_grid, 
    save_dir=f'results/{dataset}/{experiment_name}' 
)
results = exp.run_experiment(X_train, X_test, y_train, y_test)
# save
exp.save_results(results)

Evaluating NN
Data saved at results/dataset1/experiment4/results_ica.pkl


## 3. RP

In [8]:
exp = ExperimentRdNN(
    rd = 'rp',
    n_components_range = n_components_range,
    nn_params_grid=params_grid, 
    save_dir=f'results/{dataset}/{experiment_name}' 
)
results = exp.run_experiment(X_train, X_test, y_train, y_test)
# save
exp.save_results(results)

Running RD with n_components=8
Running NN
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Evaluating NN
Running RD with n_components=10
Running NN
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Evaluating NN
Running RD with n_components=12
Running NN
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Evaluating NN
Running RD with n_components=14
Running NN
Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 3/5] END hidden_layer_sizes=[10], learning_rate_init=0.001;, score=(train=0.000, test=0.000) total time=   0.3s
[CV 3/5] END hidden_layer_sizes=[10], learning_rate_init=0.0005;, score=(train=0.000, test=0.000) total time=   0.3s
[CV 3/5] END hidden_layer_sizes=[15], learning_rate_init=0.1;, score=(train=0.537, test=0.521) total time=   0.3s
[CV 1/5] END hidden_layer_sizes=[15], learning_rate_init=0.02;, score=(train=0.528, test=0.507) total time=   0.4s
[CV 5/5] END hidden_layer_sizes=[15], learning_rate_init=0.01;, score=(train=

## Evaluation

### 1. PCA

In [9]:
results = load_pickle('results/dataset1/experiment4/results_pca.pkl')
cv_result, eval_df, n = organize_experiment4_results(results)
print('the best n_components is: ', n)
print(f"the nn params for the best n component is: {cv_result[cv_result['n_components']==n]['params'].values[0]}")
print('the best evaluation result is: ', eval_df[eval_df['n_components']==n])
# plot
plot_train_val_score(
    data = cv_result,
    x_col = 'n_components',
    y_train = 'mean_train_score',
    y_val = 'mean_test_score',
    output_dir='figs',
    dataset=dataset,
    experiment=experiment_name,
    algo_name='pca'
)

the best n_components is:  22
the nn params for the best n component is: {'hidden_layer_sizes': [10, 10], 'learning_rate_init': 0.05}
the best evaluation result is:     n_components  accuracy  precision    recall        f1
7            22   0.92219   0.653084  0.645267  0.649152
Plot saved at figs/dataset1/experiment4/train_val_score_pca.png


In [10]:
# save the train and test transformed data
X_train = results[n]['X_train_transformed']
X_test = results[n]['X_test_transformed']
# save
np.save(f'data/{dataset}/pca_X_train.npy', X_train)
np.save(f'data/{dataset}/pca_X_test.npy', X_test)

### 2. ICA

In [11]:
results = load_pickle('results/dataset1/experiment4/results_ica.pkl')
cv_result, eval_df, n = organize_experiment4_results(results)
print('the best n_components is: ', n)
print(f"the nn params for the best n component is: {cv_result[cv_result['n_components']==n]['params'].values[0]}")
print('the best evaluation result is: ', eval_df[eval_df['n_components']==n])
# plot
plot_train_val_score(
    data = cv_result,
    x_col = 'n_components',
    y_train = 'mean_train_score',
    y_val = 'mean_test_score',
    output_dir='figs',
    dataset=dataset,
    experiment=experiment_name,
    algo_name='ica'
)

the best n_components is:  10
the nn params for the best n component is: {'hidden_layer_sizes': [10, 10], 'learning_rate_init': 0.1}
the best evaluation result is:     n_components  accuracy  precision    recall        f1
1            10  0.917334   0.642005  0.585419  0.612408
Plot saved at figs/dataset1/experiment4/train_val_score_ica.png


In [12]:
# save the train and test transformed data
X_train = results[n]['X_train_transformed']
X_test = results[n]['X_test_transformed']
# save
np.save(f'data/{dataset}/ica_X_train.npy', X_train)
np.save(f'data/{dataset}/ica_X_test.npy', X_test)

### 3. RP

In [13]:
results = load_pickle('results/dataset1/experiment4/results_rp.pkl')
cv_result, eval_df, n = organize_experiment4_results(results)
print('the best n_components is: ', n)
print(f"the nn params for the best n component is: {cv_result[cv_result['n_components']==n]['params'].values[0]}")
print('the best evaluation result is: ', eval_df[eval_df['n_components']==n])
# plot
plot_train_val_score(
    data = cv_result,
    x_col = 'n_components',
    y_train = 'mean_train_score',
    y_val = 'mean_test_score',
    output_dir='figs',
    dataset=dataset,
    experiment=experiment_name,
    algo_name='rp'
)



the best n_components is:  18
the nn params for the best n component is: {'hidden_layer_sizes': [15, 10], 'learning_rate_init': 0.005}
the best evaluation result is:     n_components  accuracy  precision   recall        f1
5            18  0.915756   0.656033  0.51469  0.576829
Plot saved at figs/dataset1/experiment4/train_val_score_rp.png


In [14]:
# save the train and test transformed data
X_train = results[n]['X_train_transformed']
X_test = results[n]['X_test_transformed']
# save
np.save(f'data/{dataset}/rp_X_train.npy', X_train)
np.save(f'data/{dataset}/rp_X_test.npy', X_test)

# Experiment 2
this experiment, I won't tune the nn model, instead I will use the neural netwrok setting from AI

In [44]:
X_train, X_test,y_train, y_test = load_processed_data(f'data/dataset1')
print('The training data has shape: ', X_train.shape)
print('The testing data has shape: ', X_test.shape)

The training data has shape:  (32950, 67)
The testing data has shape:  (8238, 67)


In [46]:
nn_params = {
    'hidden_layer_sizes':[20, 20],
    'learning_rate_init': 0.001,
    'max_iter': 2000, 
    'learning_rate': 'adaptive', 
    'early_stopping': True
}
rd_config = [
    {'rd': 'pca', 'n_component': 25},
    {'rd': 'ica', 'n_component': 55},
    {'rd': 'rp', 'n_component': 37}
]
exp = ExperimentNN(
    rd_config=rd_config,
    k_cluster=2,
    nn_config=nn_params,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    save_dir=f'results/dataset1/experiment4'
)
results = exp.run_experiment()
# save
exp.save_results(results)

Running base NN
Run learning curve for base NN
Running NN with pca and n_components=25
Run learning curve for pca NN


/Users/Placebo/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/Users/Placebo/anaconda3/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Running NN with ica and n_components=55
Run learning curve for ica NN
Running NN with rp and n_components=37
Run learning curve for rp NN
Data saved at results/dataset1/experiment4/a1_setting_results.pkl


In [54]:
# organize the results
results = load_pickle('results/dataset1/experiment4/a1_setting_results.pkl')
metrics_df = []
for key, value in results.items():
    metrics = value['metrics']
    metrics['rd'] = key
    metrics_df.append(metrics)
metrics_df = pd.DataFrame(metrics_df) 
metrics_df

,accuracy,precision,recall,f1,rd
0,0.917213,0.656539,0.540805,0.593079,base
1,0.918427,0.672245,0.524483,0.589242,pca
2,0.915878,0.656510,0.515778,0.577697,ica
3,0.920005,0.660099,0.583243,0.619295,rp


In [48]:
# organize the learning curve
def organize_learning_curve(results):
    df = pd.DataFrame()
    for key, value in results.items():
        lr = value['learning_curve']
        train_size = lr['train_sizes']
        train_score_mean = np.mean(lr['train_scores'], axis=1)
        train_score_std = np.std(lr['train_scores'], axis=1)
        val_score_mean = np.mean(lr['val_scores'], axis=1)
        val_score_std = np.std(lr['val_scores'], axis=1)
        fit_time_mean = np.mean(lr['fit_times'], axis=1)
        fit_time_std = np.std(lr['fit_times'], axis=1)
        # prepare data frame
        tmp_df = pd.DataFrame({
            f'{key}_train_score_mean': train_score_mean,
            f'{key}_train_score_std': train_score_std,
            f'{key}_val_score_mean': val_score_mean,
            f'{key}_val_score_std': val_score_std,
            f'{key}_fit_time_mean': fit_time_mean,
            f'{key}_fit_time_std': fit_time_std,
        })
        df = pd.concat([df, tmp_df], axis=1)
    df['train_sizes'] = train_size
    return df

df = organize_learning_curve(results)
df     
        

,base_train_score_mean,base_train_score_std,base_val_score_mean,base_val_score_std,base_fit_time_mean,base_fit_time_std,pca_train_score_mean,pca_train_score_std,pca_val_score_mean,pca_val_score_std,...,ica_val_score_std,ica_fit_time_mean,ica_fit_time_std,rp_train_score_mean,rp_train_score_std,rp_val_score_mean,rp_val_score_std,rp_fit_time_mean,rp_fit_time_std,train_sizes
0,0.915099,0.003945,0.905463,0.002285,0.135021,0.014537,0.898331,0.003794,0.896419,0.005247,...,0.000061,0.074214,0.019182,0.912064,0.000152,0.898179,0.001594,0.122938,0.020410,2636
1,0.924524,0.005416,0.907011,0.002533,0.491210,0.086340,0.913575,0.002400,0.906980,0.001305,...,0.000061,0.223060,0.044562,0.915396,0.005507,0.904613,0.001887,0.414939,0.064344,8567
2,0.914071,0.001134,0.909803,0.002062,0.515663,0.051244,0.909450,0.000838,0.906889,0.001307,...,0.000061,0.300816,0.063958,0.913754,0.001754,0.906070,0.002006,0.637221,0.158093,14498
3,0.916599,0.004506,0.908710,0.002854,0.986533,0.358662,0.908904,0.003772,0.906252,0.003467,...,0.008166,0.559378,0.217539,0.913026,0.003137,0.908225,0.001935,0.839413,0.319399,20429
4,0.919408,0.001596,0.909651,0.000565,1.177815,0.204246,0.911259,0.002148,0.907982,0.001587,...,0.008636,1.105736,0.255112,0.915971,0.001332,0.908862,0.002520,1.099564,0.213491,26360


In [63]:
plot_learning_curves(
    df=df,
    keys=['base', 'pca', 'ica', 'rp'],
    output_dir='figs',
    dataset='dataset1',
    experiment='experiment4'
)    

Plot saved at figs/dataset1/experiment4/learning_curves.png
